# Meta-analytic functional decoding
## Discrete functional decoding


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nimare
import nibabel as nib
from nilearn import datasets
from nimare.stats import pearson
from nimare.dataset import Dataset
from nimare.decode import discrete
from nimare.utils import get_resource_path
from nimare.extract import fetch_neurosynth
from nimare.io import convert_neurosynth_to_dataset
from nimare.decode import continuous
from nilearn import image
from nilearn.plotting import plot_roi
from nilearn.image import load_img
from nilearn.image import new_img_like
from nilearn import datasets, plotting
from nilearn.masking import _unmask_3d
from nilearn.maskers import nifti_spheres_masker
from nibabel import Nifti1Image


## Create a region of interest
### 创建球形ROI

根据疾病组和健康组元分析结果，创建相关的ROI

疾病组： 创建一个brain mask：
- L Frontal_Sup_Medial （-6，58，18）
- L Cingulate_Post (-2, -54, 28)

健康组： 创建五个brain mask
- L Cingulate_Ant （0，48，8）
- L OFCpost （-38，24，-14）
- L Cingulate_Post （-2，-54，26）
- L Angular（-44, -58，24）
- R Cingulate_Mid （2， -16, 38）

参考的neurostars 链接

> https://neurostars.org/t/how-to-interpret-results-of-nimare-decode-discrete-roiassociationdecoder/30190

In [ ]:
# let's assume we are in MNI space
brain_mask = datasets.load_mni152_brain_mask()

In [ ]:

# generate a mask for the left FPole
_, A = nifti_spheres_masker._apply_mask_and_get_affinity(
    seeds=[(-2, -54, 28)], 
    niimg=None,
    radius=10,
    allow_overlap=False, 
    mask_img=brain_mask)


# Unmask the result (converts the sparse matrix to a 3D array)
# _unmask_3d will convert the masked array into the shape of the original brain mask
L_Cingulate_Post_patient_mask = _unmask_3d(
    X=A.toarray().flatten(), 
    mask=brain_mask.get_fdata().astype(bool))

L_Cingulate_Post_patient_roi = Nifti1Image(L_Cingulate_Post_patient_mask, brain_mask.affine)



In [ ]:
from nilearn import plotting

# 直接使用 cmap="Oranges" 进行颜色设定
plotting.plot_roi(L_Cingulate_Post_patient_roi, cmap="Wistia")
plotting.show()


In [ ]:
L_Cingulate_Post_patient_roi.to_filename('../Data/AnalysisData/ROI/L_Cingulate_Post_patient.nii.gz')

### 解码部分

In [2]:
# get neurosynth data
databases = nimare.extract.fetch_neurosynth(data_dir='../Data')[0]

# convert to NiMARE dataset (Note: This can take a while!)
ds = nimare.io.convert_neurosynth_to_dataset(
    coordinates_file=databases['coordinates'],
    metadata_file=databases['metadata'],
    annotations_files=databases['features']
    )

INFO:nimare.extract.utils:Dataset found in ../Data/neurosynth

INFO:nimare.extract.extract:Searching for any feature files matching the following criteria: [('data-neurosynth', 'version-7')]


File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.


In [5]:
ds.coordinates

,id,study_id,contrast_id,x,y,z,space
1483,10022492-1,10022492,1,36.0,-58.0,52.0,mni152_2mm
1499,10022492-1,10022492,1,48.0,24.0,20.0,mni152_2mm
1498,10022492-1,10022492,1,-42.0,26.0,20.0,mni152_2mm
1497,10022492-1,10022492,1,-36.0,30.0,16.0,mni152_2mm
1496,10022492-1,10022492,1,-30.0,32.0,0.0,mni152_2mm
...,...,...,...,...,...,...,...
1479,9990082-1,9990082,1,42.0,-54.0,-21.0,mni152_2mm
1480,9990082-1,9990082,1,-36.0,-87.0,-6.0,mni152_2mm
1481,9990082-1,9990082,1,30.0,-81.0,-15.0,mni152_2mm
1467,9990082-1,9990082,1,-18.0,-60.0,54.0,mni152_2mm


In [6]:
print("Coordinates columns:", ds.coordinates.columns.tolist())

Coordinates columns: ['id', 'study_id', 'contrast_id', 'x', 'y', 'z', 'space']


In [ ]:
from nimare.decode import discrete

# decode ROI image

for roi_name, roi_path in roi_files.items():
    # load ROI image
    roi_img = nib.load(roi_path)
    
    # generate decoder
    decoder = discrete.ROIAssociationDecoder(
        roi_img,
        frequency_threshold=frequency_threshold,
        u=u,
        correction=correction,
    )
    
    # fit decoder to dataset
    decoder.fit(ds)
    
    # decoding and save to csv
    decoded_ds = decoder.transform()
    output_csv_path = f'../Output/6_Decoding/{roi_name}_decode.csv'
    decoded_ds.to_csv(output_csv_path, index=True)

    print(f"Decoding for {roi_name} completed and saved to {output_csv_path}")

In [ ]:
import pandas as pd
import os

# 假设您有多个 CSV 文件存放在一个文件夹中
input_folder = '../Output/6_Decoding'  # 修改为存放 CSV 文件的文件夹路径
output_folder = '../Output/6_Decoding/clean_decoding'  # 修改为保存处理后的文件的文件夹路径

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# 遍历文件夹中的所有 CSV 文件
for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):  # 仅处理 CSV 文件
        input_path = os.path.join(input_folder, file_name)
        
        # 读取 CSV 文件
        df = pd.read_csv(input_path)
        
        # 筛选以 "terms_abstract_tfidf__" 开头的行
        filtered_df = df[df['feature'].str.startswith('terms_abstract_tfidf__', na=False)]
        
        # 保存处理后的文件到输出文件夹
        output_path = os.path.join(output_folder, file_name)
        filtered_df.to_csv(output_path, index=False)
        print(f'已处理文件: {file_name} 并保存到 {output_path}')


## 使用cognitive atlas的术语进行解码

In [ ]:
import requests

# Cognitive Atlas API base URL
base_url = "https://www.cognitiveatlas.org/api/v-alpha"

# Endpoints for concepts, tasks, and disorders
concepts_endpoint = f"{base_url}/concept?format=json"
tasks_endpoint = f"{base_url}/task?format=json"
disorders_endpoint = f"{base_url}/disorder?format=json"


# Fetch concepts and tasks data
concepts_response = requests.get(concepts_endpoint)
tasks_response = requests.get(tasks_endpoint)
disorders_response = requests.get(disorders_endpoint)

# Extract names from the response data
concepts = concepts_response.json()
tasks = tasks_response.json()
disorders = disorders_response.json()

# Get the names of concepts and tasks
concept_names = [concept['name'] for concept in concepts]
task_names = [task['name'] for task in tasks]
disorder_names = [disorder['name'] for disorder in disorders]

cognitive_atlas_terms = concept_names + task_names + disorder_names

In [ ]:
terms_to_keep = ['id'] + [term for term in ds.annotations.columns if term.split('__')[-1] in cognitive_atlas_terms]
ds.annotations = ds.annotations[terms_to_keep]

In [ ]:
roi_files = {
    "L_Angular": "../Data/AnalysisData/ROI/L_Angular.nii.gz",
    "L_Cingulate_Ant": "../Data/AnalysisData/ROI/L_Cingulate_Ant.nii.gz",
    "L_Cingulate_Post_patient": "../Data/AnalysisData/ROI/L_Cingulate_Post_patient.nii.gz",
    "L_Cingulate_Post": "../Data/AnalysisData/ROI/L_Cingulate_Post.nii.gz",
    "L_Frontal_Sup_Medial": "../Data/AnalysisData/ROI/L_Frontal_Sup_Medial.nii.gz",
    "L_OFCpost": "../Data/AnalysisData/ROI/L_OFCpost.nii.gz",
    "R_Cingulate_Mid": "../Data/AnalysisData/ROI/R_Cingulate_Mid.nii.gz"
}

In [ ]:
L_Angular = "../Data/AnalysisData/ROI/L_Angular.nii.gz"


In [ ]:
L_Cingulate_Ant= "/Users/ss/Documents/Self_Psych_Meta/Data/AnalysisData/ROI/L_Cingulate_Ant.nii.gz",


In [ ]:
L_Cingulate_Post_patient = "../Data/AnalysisData/ROI/L_Cingulate_Post_patient.nii.gz",
L_Cingulate_Post ="../Data/AnalysisData/ROI/L_Cingulate_Post.nii.gz",
L_Frontal_Sup_Medial = "../Data/AnalysisData/ROI/L_Frontal_Sup_Medial.nii.gz",
L_OFCpost = "../Data/AnalysisData/ROI/L_OFCpost.nii.gz",
R_Cingulate_Mid = "../Data/AnalysisData/ROI/R_Cingulate_Mid.nii.gz"

In [ ]:
# define decoder parameters
frequency_threshold = 0.001
u = 0.05
correction = "fdr_bh"

In [ ]:
from nimare.decode.continuous import CorrelationDecoder
from nimare.meta.cbma import mkda

In [ ]:
decoder = CorrelationDecoder(L_Cingulate_Ant, 
                             frequency_threshold=0.001, 
                             meta_estimator=mkda.MKDAChi2,
                             target_image='z_desc-association')
decoder.fit(ds)
decoded_df = decoder.transform()

In [ ]:
decoded_df.to_csv('/Users/ss/Documents/Self_Psych_Meta/Output/6_Decoding/L_Angular.csv', index=True)

### Plot word cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib

# 定义输入文件列表
csv_files = [
    '../results/extract_decoding/extract_Angular_decode.csv',
    '../results/extract_decoding/extract_Cingulate_decode.csv',
    '../results/extract_decoding/extract_FPole_decode.csv',
    '../results/extract_decoding/extract_L_FOC_decode.csv',
    '../results/extract_decoding/extract_R_FOC_decode.csv',
    '../results/extract_decoding/extract_PCG_decode.csv'
]

# 定义输出目录
output_dir = '../results/wordcloud'
os.makedirs(output_dir, exist_ok=True)  # 如果目录不存在，则创建


In [ ]:
# 批量处理每个 CSV 文件
# "viridis" 绿色
for file_path in csv_files:
    # 读取 CSV 数据
    data = pd.read_csv(file_path)

    # 构建词频字典，使用 'cognitive_feature' 作为单词，'r' 作为词频
    word_frequencies = data.set_index('cognitive_feature')['r'].to_dict()

    # 创建词云
    wordcloud = WordCloud(
        width=800,
        height=400,
        colormap="YlOrRd",
        background_color='white',
        prefer_horizontal=1.0
    ).generate_from_frequencies(word_frequencies)

    # 绘制并显示词云图
    plt.figure(figsize=(8, 4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # 关闭坐标轴
    plt.show()

    # 保存词云图为文件
    file_name = os.path.basename(file_path).replace('extract_', '').replace('_decode.csv', '')  # 提取文件名的一部分
    output_path = os.path.join(output_dir, f'{file_name}_wordcloud.png')
    wordcloud.to_file(output_path)

    print(f"词云图已保存到 {output_path}")
